# Challenge 2: Sentiment Analysis

In this challenge we will learn sentiment analysis and practice performing sentiment analysis on Twitter tweets. 

## Introduction

Sentiment analysis is to *systematically identify, extract, quantify, and study affective states and subjective information* based on texts ([reference](https://en.wikipedia.org/wiki/Sentiment_analysis)). In simple words, it's to understand whether a person is happy or unhappy in producing the piece of text. Why we (or rather, companies) care about sentiment in texts? It's because by understanding the sentiments in texts, we will be able to know if our customers are happy or unhappy about our products and services. If they are unhappy, the subsequent action is to figure out what have caused the unhappiness and make improvements.

Basic sentiment analysis only understands the *positive* or *negative* (sometimes *neutral* too) polarities of the sentiment. More advanced sentiment analysis will also consider dimensions such as agreement, subjectivity, confidence, irony, and so on. In this challenge we will conduct the basic positive vs negative sentiment analysis based on real Twitter tweets.

NLTK comes with a [sentiment analysis package](https://www.nltk.org/api/nltk.sentiment.html). This package is great for dummies to perform sentiment analysis because it requires only the textual data to make predictions. For example:

```python
>>> from nltk.sentiment.vader import SentimentIntensityAnalyzer
>>> txt = "Ironhack is a Global Tech School ranked num 2 worldwide.   Our mission is to help people transform their careers and join a thriving community of tech professionals that love what they do."
>>> analyzer = SentimentIntensityAnalyzer()
>>> analyzer.polarity_scores(txt)
{'neg': 0.0, 'neu': 0.741, 'pos': 0.259, 'compound': 0.8442}
```

In this challenge, however, you will not use NLTK's sentiment analysis package because in your Machine Learning training in the past 2 weeks you have learned how to make predictions more accurate than that. The [tweets data](https://www.kaggle.com/kazanova/sentiment140) we will be using today are already coded for the positive/negative sentiment. You will be able to use the Naïve Bayes classifier you learned in the lesson to predict the sentiment of tweets based on the labels.

## Conducting Sentiment Analysis

### Loading and Exploring Data

The dataset we'll be using today is located on Kaggle (https://www.kaggle.com/kazanova/sentiment140). Once you have downloaded and imported the dataset, it you will need to define the columns names: df.columns = ['target','id','date','flag','user','text']

*Notes:* 

* The dataset is huuuuge (1.6m tweets). When you develop your data analysis codes, you can sample a subset of the data (e.g. 20k records) so that you will save a lot of time when you test your codes.

In [1]:
import pandas as pd
import re
import nltk

from nltk.classify import NaiveBayesClassifier
from sklearn.model_selection import train_test_split
from nltk.probability import FreqDist
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import stopwords

In [2]:
df = pd.read_csv('training.1600000.processed.noemoticon.csv', encoding='latin-1')

In [3]:
df.columns = ['target', 'id', 'date', 'flag', 'user', 'text']
df.head()

df['text'].head()

0    is upset that he can't update his Facebook by ...
1    @Kenichan I dived many times for the ball. Man...
2      my whole body feels itchy and like its on fire 
3    @nationwideclass no, it's not behaving at all....
4                        @Kwesidei not the whole crew 
Name: text, dtype: object

In [4]:
print(df.columns)

Index(['target', 'id', 'date', 'flag', 'user', 'text'], dtype='object')


### Prepare Textual Data for Sentiment Analysis

Now, apply the functions you have written in Challenge 1 to your whole data set. These functions include:

* `clean_up()`

* `tokenize()`

* `stem_and_lemmatize()`

* `remove_stopwords()`

Create a new column called `text_processed` in the dataframe to contain the processed data. At the end, your `text_processed` column should contain lists of word tokens that are cleaned up. Your data should look like below:

![Processed Data](data-cleaning-results.png)

In [5]:
# FUNCTIONS

def clean_up(text):
    text = re.sub(r'\b(?:https?|ftp)://\S+', '', text)
    text = re.sub(r'[^A-Za-z\s]', ' ', text).lower()
    text = re.sub(r'\s+', ' ', text).strip()
    return text

def tokenize(text):
    tokens = nltk.word_tokenize(text)
    return tokens

def stem_and_lemmatize(words):
    stemmer = PorterStemmer()
    lemmatizer = WordNetLemmatizer()
    stemmed_and_lemmatized = [lemmatizer.lemmatize(stemmer.stem(word)) for word in words]
    return stemmed_and_lemmatized

def remove_stopwords(words):
    stop_words = set(stopwords.words('english'))
    processed_words = [word for word in words if word.lower() not in stop_words]
    return processed_words

df['cleaned_text'] = df['text'].apply(clean_up)
df['tokenized_text'] = df['cleaned_text'].apply(tokenize)
df['stem_lemma_text'] = df['tokenized_text'].apply(stem_and_lemmatize)
df['text_processed'] = df['stem_lemma_text'].apply(remove_stopwords)


In [6]:
df[['text', 'text_processed']].head()

,text,text_processed
0,is upset that he can't update his Facebook by ...,"[upset, updat, hi, facebook, text, might, cri,..."
1,@Kenichan I dived many times for the ball. Man...,"[kenichan, dive, mani, time, ball, manag, save..."
2,my whole body feels itchy and like its on fire,"[whole, bodi, feel, itchi, like, fire]"
3,"@nationwideclass no, it's not behaving at all....","[nationwideclass, behav, mad, whi, becaus, see]"
4,@Kwesidei not the whole crew,"[kwesidei, whole, crew]"


### Creating Bag of Words

The purpose of this step is to create a [bag of words](https://en.wikipedia.org/wiki/Bag-of-words_model) from the processed data. The bag of words contains all the unique words in your whole text body (a.k.a. *corpus*) with the number of occurrence of each word. It will allow you to understand which words are the most important features across the whole corpus.

Also, you can imagine you will have a massive set of words. The less important words (i.e. those of very low number of occurrence) do not contribute much to the sentiment. Therefore, you only need to use the most important words to build your feature set in the next step. In our case, we will use the top 5,000 words with the highest frequency to build the features.

In the cell below, combine all the words in `text_processed` and calculate the frequency distribution of all words. A convenient library to calculate the term frequency distribution is NLTK's `FreqDist` class ([documentation](https://www.nltk.org/api/nltk.html#module-nltk.probability)). Then select the top 5,000 words from the frequency distribution.

In [7]:
all_words = [word for words in df['text_processed'] for word in words]
freq_dist = FreqDist(all_words)
top_words = freq_dist.most_common(1000)

In [8]:
display(top_words)

[('go', 138778),
 ('get', 110865),
 ('day', 109399),
 ('wa', 105044),
 ('thi', 94810),
 ('good', 92591),
 ('work', 87886),
 ('like', 83850),
 ('love', 83115),
 ('quot', 73388),
 ('u', 71319),
 ('got', 71118),
 ('today', 68713),
 ('time', 66420),
 ('lol', 59484),
 ('thank', 59442),
 ('miss', 58028),
 ('one', 57619),
 ('back', 57419),
 ('want', 57354),
 ('know', 54977),
 ('see', 51387),
 ('feel', 51268),
 ('think', 51213),
 ('im', 50789),
 ('realli', 50084),
 ('amp', 48768),
 ('night', 45829),
 ('hope', 44956),
 ('watch', 43649),
 ('still', 43624),
 ('need', 43449),
 ('make', 43222),
 ('well', 42968),
 ('new', 42479),
 ('na', 42180),
 ('home', 40849),
 ('oh', 40051),
 ('look', 39760),
 ('ha', 39528),
 ('come', 39136),
 ('much', 37068),
 ('last', 36275),
 ('twitter', 36201),
 ('morn', 35732),
 ('tomorrow', 34339),
 ('wish', 34043),
 ('great', 33695),
 ('wait', 32605),
 ('sleep', 32475),
 ('haha', 31543),
 ('sad', 30046),
 ('fun', 28735),
 ('whi', 28539),
 ('week', 28342),
 ('tri', 28271),

### Building Features

Now let's build the features. Using the top 5,000 words, create a 2-dimensional matrix to record whether each of those words is contained in each document (tweet). Then you also have an output column to indicate whether the sentiment in each tweet is positive. For example, assuming your bag of words has 5 items (`['one', 'two', 'three', 'four', 'five']`) out of 4 documents (`['A', 'B', 'C', 'D']`), your feature set is essentially:

| Doc | one | two | three | four | five | is_positive |
|---|---|---|---|---|---|---|
| A | True | False | False | True | False | True |
| B | False | False | False | True | True | False |
| C | False | True | False | False | False | True |
| D | True | False | False | False | True | False|

However, because the `nltk.NaiveBayesClassifier.train` class we will use in the next step does not work with Pandas dataframe, the structure of your feature set should be converted to the Python list looking like below:

```python
[
	({
		'one': True,
		'two': False,
		'three': False,
		'four': True,
		'five': False
	}, True),
	({
		'one': False,
		'two': False,
		'three': False,
		'four': True,
		'five': True
	}, False),
	({
		'one': False,
		'two': True,
		'three': False,
		'four': False,
		'five': False
	}, True),
	({
		'one': True,
		'two': False,
		'three': False,
		'four': False,
		'five': True
	}, False)
]
```

To help you in this step, watch the [following video](https://www.youtube.com/watch?v=-vVskDsHcVc) to learn how to build the feature set with Python and NLTK. The source code in this video can be found [here](https://pythonprogramming.net/words-as-features-nltk-tutorial/).

[![Building Features](building-features.jpg)](https://www.youtube.com/watch?v=-vVskDsHcVc)

In [9]:
tokenized_texts = [tokenize(text) for text in df['cleaned_text']]
features = [{word: (word in tokens) for word in top_words} for tokens in tokenized_texts]

for feature in features:
    print(feature)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [16]:
labels = ['value'] * len(features)
features_list = list(zip(features, labels))

print(features[:5])
print(features_list[:5])

[{('go', 138778): False, ('get', 110865): False, ('day', 109399): False, ('wa', 105044): False, ('thi', 94810): False, ('good', 92591): False, ('work', 87886): False, ('like', 83850): False, ('love', 83115): False, ('quot', 73388): False, ('u', 71319): False, ('got', 71118): False, ('today', 68713): False, ('time', 66420): False, ('lol', 59484): False, ('thank', 59442): False, ('miss', 58028): False, ('one', 57619): False, ('back', 57419): False, ('want', 57354): False, ('know', 54977): False, ('see', 51387): False, ('feel', 51268): False, ('think', 51213): False, ('im', 50789): False, ('realli', 50084): False, ('amp', 48768): False, ('night', 45829): False, ('hope', 44956): False, ('watch', 43649): False, ('still', 43624): False, ('need', 43449): False, ('make', 43222): False, ('well', 42968): False, ('new', 42479): False, ('na', 42180): False, ('home', 40849): False, ('oh', 40051): False, ('look', 39760): False, ('ha', 39528): False, ('come', 39136): False, ('much', 37068): False, ('

### Building and Traininng Naive Bayes Model

In this step you will split your feature set into a training and a test set. Then you will create a Bayes classifier instance using `nltk.NaiveBayesClassifier.train` ([example](https://www.nltk.org/book/ch06.html)) to train with the training dataset.

After training the model, call `classifier.show_most_informative_features()` to inspect the most important features. The output will look like:

```
Most Informative Features
	    snow = True            False : True   =     34.3 : 1.0
	  easter = True            False : True   =     26.2 : 1.0
	 headach = True            False : True   =     20.9 : 1.0
	    argh = True            False : True   =     17.6 : 1.0
	unfortun = True            False : True   =     16.9 : 1.0
	    jona = True             True : False  =     16.2 : 1.0
	     ach = True            False : True   =     14.9 : 1.0
	     sad = True            False : True   =     13.0 : 1.0
	  parent = True            False : True   =     12.9 : 1.0
	  spring = True            False : True   =     12.7 : 1.0
```

The [following video](https://www.youtube.com/watch?v=rISOsUaTrO4) will help you complete this step. The source code in this video can be found [here](https://pythonprogramming.net/naive-bayes-classifier-nltk-tutorial/).

[![Building and Training NB](nb-model-building.jpg)](https://www.youtube.com/watch?v=rISOsUaTrO4)

In [ ]:
# SPLIT TRAIN/TEST

train_set, test_set = train_test_split(features, test_size=0.2, random_state=42)

classifier = NaiveBayesClassifier.train(train_set)
classifier.show_most_informative_features(30) 


### Testing Naive Bayes Model

Now we'll test our classifier with the test dataset. This is done by calling `nltk.classify.accuracy(classifier, test)`.

As mentioned in one of the tutorial videos, a Naive Bayes model is considered OK if your accuracy score is over 0.6. If your accuracy score is over 0.7, you've done a great job!

In [18]:
# TEST

accuracy = nltk.classify.accuracy(classifier, test_set)

print(f'Classifier Accuracy: {accuracy * 100:.2f}%')

Classifier Accuracy: 100.00%


## Bonus Question 1: Improve Model Performance

If you are still not exhausted so far and want to dig deeper, try to improve your classifier performance. There are many aspects you can dig into, for example:

* Improve stemming and lemmatization. Inspect your bag of words and the most important features. Are there any words you should furuther remove from analysis? You can append these words to further remove to the stop words list.

* Remember we only used the top 5,000 features to build model? Try using different numbers of top features. The bottom line is to use as few features as you can without compromising your model performance. The fewer features you select into your model, the faster your model is trained. Then you can use a larger sample size to improve your model accuracy score.

In [ ]:
# your code here

## Bonus Question 2: Machine Learning Pipeline

In a new Jupyter Notebook, combine all your codes into a function (or a class). Your new function will execute the complete machine learning pipeline job by receiving the dataset location and output the classifier. This will allow you to use your function to predict the sentiment of any tweet in real time. 

In [ ]:
# your code here

## Bonus Question 3: Apache Spark

If you have completed the Apache Spark advanced topic lab, what you can do is to migrate your pipeline from local to a Databricks Notebook. Share your notebook with your instructor and classmates to show off your achievements!

In [ ]:
# your code here